# Imports

In [769]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']
sys.prefix = '/home/joaom/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

# Load Data

In [805]:
from src import utils
from datetime import datetime

In [806]:
conn = utils.connect_athena(path='../configs/athena.yaml')

In [807]:
# cities = ['São Paulo', 'Rio de Janeiro', 'Campinas']
# cities = "'"+"','".join(cities)+"'"

In [809]:
df = pd.read_sql_query("""
select 
    *
from spd_sdv_waze_corona.pipeline_test_daily_diff
""", conn)

In [810]:
daily = pd.read_sql_query(f"""
select 
   *
from spd_sdv_waze_corona.pipeline_test_daily_daily
where city in ({cities})
""", conn)

In [1280]:
historical = pd.read_sql_query(f"""
select 
    *
from spd_sdv_waze_corona.pipeline_test_historical_historical
where year = 2019
""", conn)

In [1281]:
daily = pd.read_sql_query(f"""
select 
    *
from spd_sdv_waze_corona.pipeline_test_historical_historical
where year = 2020
""", conn)

In [1276]:
cities = pd.read_sql_query(f"""
select 
    country, city, count(*) counta
from spd_sdv_waze_corona.pipeline_test_historical_historical_raw
where year = 2020
group by country, city
""", conn)

In [1279]:
cities.to_csv(RAW_PATH / 'waze_cities.csv')

In [1036]:
daily.head()

,country,country_iso,city,timezone,year,month,dow,day,hour,level,sum_length,city_slug
0,Perú,PE,Lima,America/Lima,2020,1,2,28,17,3,2516980,lima
1,Perú,PE,Lima,America/Lima,2020,3,4,12,3,2,541295,lima
2,Perú,PE,Lima,America/Lima,2020,2,6,29,16,4,1577080,lima
3,Perú,PE,Lima,America/Lima,2020,3,6,14,20,4,689532,lima
4,Perú,PE,Lima,America/Lima,2020,1,2,28,22,4,1672225,lima


In [1037]:
historical.head()

,country,country_iso,city,timezone,year,month,dow,day,hour,level,sum_length,city_slug
0,Brasil,BR,Campinas,America/Sao_Paulo,2019,1,5,18,22,4,38627,campinas
1,Brasil,BR,Campinas,America/Sao_Paulo,2019,1,5,18,18,4,84772,campinas
2,Brasil,BR,Campinas,America/Sao_Paulo,2019,1,4,31,21,3,195596,campinas
3,Brasil,BR,Campinas,America/Sao_Paulo,2019,1,4,17,20,4,114610,campinas
4,Brasil,BR,Campinas,America/Sao_Paulo,2019,2,6,2,11,3,93501,campinas


In [1078]:
holidays = yaml.load("""
- city: São Paulo
  holidays:
      - 2020-02-21
      - 2020-02-22
      - 2020-02-23
      - 2020-02-24
      - 2020-02-25
      - 2020-02-26
      - 2020-02-27
      - 2020-01-24
      - 2020-02-12
      - 2020-02-19
      - 2019-01-25
      - 2019-01-26
      - 2019-01-27
      - 2019-03-01
      - 2019-03-02
      - 2019-03-03
      - 2019-03-04
      - 2019-03-05
      - 2019-03-06
- city: Rio de Janeiro
  holidays:
      - 2020-01-20
      - 2020-01-19
      - 2020-01-18
      - 2020-02-21
      - 2020-02-22
      - 2020-02-23
      - 2020-02-24
      - 2020-02-25
      - 2020-02-26
      - 2020-02-27
      - 2020-02-28
      - 2020-02-29
      - 2020-02-12
      - 2020-02-19
      - 2019-03-01
      - 2019-03-02
      - 2019-03-03
      - 2019-03-04
      - 2019-03-05
      - 2019-03-06
- city: Campinas
  holidays: 
      - 2020-02-21
      - 2020-02-22
      - 2020-02-23
      - 2020-02-24
      - 2020-02-25
      - 2020-02-26
      - 2020-02-27
      - 2020-02-12
      - 2020-02-19
      - 2019-03-01
      - 2019-03-02
      - 2019-03-03
      - 2019-03-04
      - 2019-03-05
      - 2019-03-06
- city: Plano Piloto
  holidays: 
      - 2020-02-21
      - 2020-02-22
      - 2020-02-23
      - 2020-02-24
      - 2020-02-25
      - 2020-02-26
      - 2020-02-27
      - 2020-02-12
      - 2020-02-19
      - 2019-03-01
      - 2019-03-02
      - 2019-03-03
      - 2019-03-04
      - 2019-03-05
      - 2019-03-06
- city: Curitiba
  holidays: 
      - 2020-02-21
      - 2020-02-22
      - 2020-02-23
      - 2020-02-24
      - 2020-02-25
      - 2020-02-26
      - 2020-02-27
      - 2020-02-12
      - 2020-02-19
      - 2019-03-01
      - 2019-03-02
      - 2019-03-03
      - 2019-03-04
      - 2019-03-05
      - 2019-03-06
- city: Bogotá
  holidays:
      - 2020-01-06
      - 2019-01-07
      - 2020-02-12
      - 2020-02-19
- city: Zona Urbana Medellín
  holidays:
      - 2020-01-06
      - 2019-01-07
      - 2020-02-12
      - 2020-02-19
- city: Buenos Aires
  holidays:
      - 2020-02-22
      - 2020-02-23
      - 2020-02-24
      - 2020-02-25
      - 2019-02-22
      - 2019-02-23
      - 2019-02-24
      - 2019-02-25
      - 2020-02-12
      - 2020-02-19
- city: Lima
  holidays: 
      - 2020-02-12
      - 2020-02-19
""")

In [1079]:
def select_city(city):
    l = [h for h in holidays if h['city'] == city]
    if len(l):
        return l[0]['holidays']
    else:
        print(city)
        return []

holidays = pd.read_csv(RAW_PATH / 'holidays.csv')
holidays['date.iso'] = holidays['date.iso'].apply(pd.to_datetime)

In [1080]:
def filter_holiday(_df):
    return _df[_df[['city', 'datetime']].apply(lambda x: x['datetime'].to_pydatetime().date() 
                                       not in select_city(x['city']), 1)]

In [1081]:
def to_time(x, year):
    return datetime.strptime(f'{year}-{x["month"]}-{x["day"]}-{x["hour"]:02}', '%Y-%m-%d-%H')

In [1282]:
historical['datetime'] = historical.apply(lambda x: to_time(x, 2019), 1)
daily['datetime'] = daily.apply(lambda x: to_time(x, 2020), 1)
daily['workday'] = pd.cut(daily['dow'], [0, 5, 7], labels=[True, False])
historical['workday'] = pd.cut(historical['dow'], [0, 5, 7], labels=[True, False])

In [1283]:
daily = filter_holiday(daily)
historical = filter_holiday(historical)

Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horiz

Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia 

Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quito
Quit

Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de S

Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horiz

...

Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horiz

Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horiz

Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horiz

Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horizonte
Belo Horiz

Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia de Santiago
Provincia 

In [1084]:
daily = daily[daily['datetime'] > '2020-01-10']
historical = historical[historical['datetime'] > '2019-01-10']

In [1163]:
daily.pivot_table(index='datetime', columns='city', values='sum_length')\
.resample('D').sum()\
.iplot(theme='custom',
          title=f'Total Jam Length per Day in 2020 (every 5 minutes)',
          yTitle='meters', 
          kind='bar'
          )

In [1086]:
historical[historical['datetime'] < "2019-03-15"].pivot_table(index='datetime', columns='city', values='sum_length')\
.resample('D').sum()\
.iplot(theme='custom',
          title=f'Total Jam Length per Day in 2019 (every 5 minutes)',
          yTitle='meters', 
          )

# Compare 

In [1160]:
def get_days(_df):
    _df = _df.groupby(['city', 'month', 'dow', 'day', 'workday', 'hour'])['sum_length'].sum().reset_index()
    _df = _df.groupby(['city', 'month', 'dow', 'day', 'workday'])['sum_length'].agg(['count', 'sum'])
    indexs = list(_df.index.names )
    _df = _df.reset_index()
    return _df[(_df['count'] == 24) & (_df['month'] < 3)]\
        .sort_values(by=['city','month', 'day'])[['count', 'sum'] + indexs]

In [1161]:
def get_sum(_df):
    _df = get_days(_df)
    return _df.groupby(['city', 'dow']).mean()['sum']

In [1162]:
adjustment = pd.concat([get_sum(daily), get_sum(historical)], 1)
adjustment.columns = ['2020', '2019']
adjustment['2020/2019'] = adjustment['2020'] / adjustment['2019']
adjustment['2020 - 2019'] = adjustment['2020'] - adjustment['2019']
adjustment

,,2020,2019,2020/2019,2020 - 2019
city,dow,,,,


In [1194]:
import pytz

In [1243]:
def to_timezone(_df):
    
    return _df.apply(lambda x: x['datetime'].replace(tzinfo=pytz.utc)\
            .astimezone(pytz.timezone(x['timezone'])).replace(tzinfo=None), 1)

In [1284]:
daily['datetime'] = to_timezone(daily)
historical['datetime'] = to_timezone(historical)

In [1285]:
daily['daytime'] = daily['datetime'].apply(lambda x: x.hour > 6 & x.hour < 20).replace(to_replace={True: 'daytime', False: 'nighttime'})
historical['daytime'] = historical['datetime'].apply(lambda x: x.hour > 6 & x.hour < 20).replace(to_replace={True: 'daytime', False: 'nighttime'})

In [1286]:
expected = daily[(daily['datetime'] > '2020-03-02') &
           (daily['datetime'] < '2020-03-10')]\
    .groupby(['city', 'dow'])['sum_length'].sum().reset_index()

In [1287]:
expected = historical[(historical['datetime'] >= '2019-03-07') &
           (historical['datetime'] <= '2019-04-30')]\
    .groupby(['city', 'dow', 'day'])['sum_length'].sum().reset_index()\
    .groupby(['city', 'dow'])['sum_length'].mean().reset_index()

In [1323]:
expected = daily[(daily['datetime'] <= '2020-03-08')]\
    .groupby(['city', 'dow'])['sum_length'].sum().reset_index()

In [1324]:
# g = historical.query('month < 3').groupby(['city', 'month', 'dow', 'day', 'workday'])[['sum_length']].sum().reset_index()
# expected = g.groupby(['city', 'dow'])[['sum_length']].mean().reset_index()

seen = daily[(daily['datetime'] >= '2020-03-09')]\
             .groupby(['city', 'month', 'dow', 'day' ])[['sum_length']].sum().reset_index()

df = expected.merge(seen, on=['city', 'dow'], suffixes=('_expected', '_seen'), how='left')
# df = df.merge(adjustment.reset_index(), on=['city', 'dow'])

df['ratio'] = df['sum_length_seen'] / df['sum_length_expected']  * 100

df['date'] = df.dropna(subset=['month', 'day']).apply(lambda x: datetime.strptime(f'2020-{int(x["month"])}-{int(x["day"]):02}', '%Y-%m-%d'), 1)

In [1327]:
df[['city', 'date', 'dow', 'month', 'day', 'ratio']].to_csv(OUTPUT_PATH / 'index_example.csv')

In [1328]:
for c in df['city'].unique():

    df.query(f'city == "{c}"')\
    .pivot_table(index='date', columns='daytime', values='ratio')\
    .iplot(theme='custom', title=iplottitle(f'Traffic Variation from 1st Week March          {c}'),
          yTitle='',
               asImage=True
          )

KeyError: 'daytime'

# Compare same year previous weeks

In [141]:
ref = daily[(daily['datetime'] < '2020-03-07') & (daily['datetime'] > '2020-02-01')]

In [142]:
ref = ref.groupby(['city', 'dow', 'hour'])[['sum_length']].median().reset_index()

In [143]:
ref['datetime'] = ref.apply(lambda x: datetime.strptime(f'{x["dow"]}-{x["hour"]:02}', '%d-%H'), 1)

In [144]:
ref.pivot_table(index='datetime', columns='city', values='sum_length')\
     .iplot(theme='custom',
          title=f'Mean Jam Length per Hour per Weekday for Month {m}',
          yTitle='meters', 
          )

In [147]:
comp = daily[(daily['datetime'] > '2020-03-07')]

In [148]:
comp['datetime'] = comp.apply(lambda x: datetime.strptime(f'{x["dow"]}-{x["hour"]:02}', '%d-%H'), 1)

In [149]:
comp.pivot_table(index='datetime', columns='city', values='sum_length')\
     .iplot(theme='custom',
          title=f'Mean Jam Length per Hour per Weekday for Month {m}',
          yTitle='meters', 
          )

In [34]:
df['historical'] = df.apply(lambda x: datetime.strptime(f'{x["month"]}-{x["dow"]}-{x["hour"]:02}', '%m-%d-%H'), 1)

In [48]:
df['daily'] = df.apply(lambda x: datetime.strptime(f'2020-{x["month"]}-{x["day"]}-{x["hour"]:02}', '%Y-%m-%d-%H'), 1)

In [56]:
daily = df[['daily', 'city', 'day_length', 'month']]

daily.pivot_table(index='daily', columns='city', values='day_length')\
    .iplot(theme='custom',
          title=f'Mean Jam Length per Hour per Weekday for Month {m}',
          yTitle='meters', 
          )

In [42]:
hist = df[['historical', 'city', 'avg_length', 'month']].drop_duplicates()

for m in hist['month'].unique():
    hist.query(f'month == {m}')\
    .pivot_table(index='historical', columns='city', values='avg_length')\
    .iplot(theme='custom',
          title=f'Mean Jam Length per Hour per Weekday for Month {m}',
          yTitle='meters'
          )